# Searching datasets

erddapy can wrap the same form-like search capabilities of ERDDAP with the
_search_for_ keyword.


In [ ]:
from erddapy import ERDDAP

e = ERDDAP(
    server="https://pae-paha.pacioos.hawaii.edu/erddap",
    protocol="griddap",
)

Single word search.


In [ ]:
import pandas as pd

search_for = "etopo"

url = e.get_search_url(search_for=search_for, response="csv")

pd.read_csv(url)["Dataset ID"]

Filtering the search with extra words.


In [ ]:
search_for = "etopo5"

url = e.get_search_url(search_for=search_for, response="csv")

pd.read_csv(url)["Dataset ID"]

Filtering the search with words that should **not** be found.


In [ ]:
search_for = "etopo5 -lon360"

url = e.get_search_url(search_for=search_for, response="csv")

pd.read_csv(url)["Dataset ID"]

Quoted search or "phrase search," first let us try the unquoted search.


In [ ]:
search_for = "ocean bathymetry"

url = e.get_search_url(search_for=search_for, response="csv")

len(pd.read_csv(url)["Dataset ID"])

Too many datasets because wind, speed, and wind speed are matched. Now let's use
the quoted search to reduce the number of results to only wind speed.


In [ ]:
search_for = '"ocean bathymetry"'

url = e.get_search_url(search_for=search_for, response="csv")

len(pd.read_csv(url)["Dataset ID"])

Another common search operation would be to search multiple servers instead of
only one. In erddapy we can achieve that with `search_servers`:

In [ ]:
from erddapy.multiple_server_search import search_servers

df = search_servers(
    query="glider",
    servers_list=None,
    parallel=True,
    protocol="tabledap",
)

In [ ]:
print(f"There are {len(df)} entries in this search!")

These are the servers that have glider data according to our query.


In [ ]:
set(df["Server url"])

One way to reduce is to search a subset of the servers with the `servers_list`
argument. We can also use it to search servers that are not part of the awesome
ERDDAP list (https://github.com/IrishMarineInstitute/awesome-erddap).

One can also perform an advanced search with ERDDAP constraints
`advanced_search_servers`.


In [ ]:
from erddapy.multiple_server_search import advanced_search_servers

min_time = "2017-07-01T00:00:00Z"
max_time = "2017-09-01T00:00:00Z"
min_lon, max_lon = -127, -123.75
min_lat, max_lat = 43, 48
standard_name = "sea_water_practical_salinity"


kw = {
    "standard_name": standard_name,
    "min_lon": min_lon,
    "max_lon": max_lon,
    "min_lat": min_lat,
    "max_lat": max_lat,
    "min_time": min_time,
    "max_time": max_time,
    "cdm_data_type": "timeseries",  # let's exclude AUV's tracks
}


servers = {
    "ooi": "https://erddap.dataexplorer.oceanobservatories.org/erddap/",
    "ioos": "https://erddap.sensors.ioos.us/erddap/",
}


df = advanced_search_servers(servers_list=servers.values(), **kw)

df.head()